In [30]:
import pandas as pd
import numpy as np
import json

In [31]:
df = pd.read_csv('ep187_electrodes.csv')
position = df['position'].to_numpy()
print(json.loads(position[0]))

[138.115783691406, 166.815368652344, 110.192565917969]


In [32]:
x = []
y = []
z = []

In [33]:
for index, row in df.iterrows():
    pos = json.loads(row.position)
    x.append(pos[0])
    y.append(pos[1])
    z.append(pos[2])

In [34]:
df['x'] = x
df['y'] = y
df['z'] = z

In [35]:
df

,electrode_number,E_EEG,E_Brain,position,position_marker,x,y,z
0,11,E11,PS1,"[138.115783691406,166.815368652344,110.1925659...",Y,138.115784,166.815369,110.192566
1,12,E12,PS2,"[142.168121337891,181.290649414063,112.6060028...",Y,142.168121,181.290649,112.606003
2,13,E13,PS3,"[141.510070800781,183.282073974609,121.5120697...",Y,141.510071,183.282074,121.512070
3,19,E19,AT1,"[134.525161743164,137.630401611328,79.41387939...",Y,134.525162,137.630402,79.413879
4,20,E20,AT2,"[139.903259277344,145.456604003906,83.08189392...",Y,139.903259,145.456604,83.081894
...,...,...,...,...,...,...,...,...
79,100,E100,FP60,"[125.119987487793,126.575744628906,176.0247192...",Y,125.119987,126.575745,176.024719
80,101,E101,FP61,"[130.231506347656,114.293052673340,183.0526428...",Y,130.231506,114.293053,183.052643
81,102,E102,FP62,"[135.048889160156,103.178604125977,186.3481140...",Y,135.048889,103.178604,186.348114
82,103,E103,FP63,"[138.054092407227,90.9049072265625,185.1483001...",Y,138.054092,90.904907,185.148300


In [36]:
df.to_csv('coordinateExtacted.csv')